In [43]:
import pygmo as pg
import numpy as np
import pandas as pd

In [66]:
results = []

funcs = [1, 2, 6, 7, 9, 14]
dimensions = [2, 10, 30]
POP_SIZE = 20
N_RUNS = 51

dimensions = [2, 10]

In [67]:
for nx in dimensions:
    for func in funcs:
        generations = nx * 10000 // POP_SIZE - 1
        
        prob = pg.problem(pg.cec2014(prob_id=func, dim=nx))
        algo = pg.algorithm(pg.xnes(gen=generations, ftol=1e-9, xtol=1e-9))
        archi = pg.archipelago(N_RUNS, algo=algo, prob=prob, pop_size=POP_SIZE)
        archi.evolve(1)
        archi.wait()
        
        champions = [isl.get_population().champion_f[0] for isl in archi]
        evals = [isl.get_population().problem.get_fevals() for isl in archi]
        for run, properties in enumerate(zip(champions, evals)):
            result = {
                         "function": func,
                         "dimensions": nx,
                         "run": run,
                         "champion_f": properties[0],
                         "evals": properties[1]
                     }
            
            results.append(result)

In [68]:
results = pd.DataFrame(results, columns=["function", "dimensions", "run", "champion_f", "evals"])

In [69]:
results["error"] = results["champion_f"] - (results["function"] * 100)
results["success"] = results["error"] < 1e-8

In [64]:
results.tail(5)

,function,dimensions,run,champion_f,evals,error,success
607,14,10,46,1400.499920,10880,0.499920,False
608,14,10,47,1400.499951,40440,0.499951,False
609,14,10,48,1400.376528,100000,0.376528,False
610,14,10,49,1400.215826,16320,0.215826,False
611,14,10,50,1400.499884,15420,0.499884,False


In [62]:
pd.DataFrame(results.groupby(["function", "dimensions"]).mean().loc[:, "success"])

success
function dimensions          
1        2           1.000000
         10          1.000000
2        2           1.000000
         10          1.000000
6        2           0.980392
         10          0.568627
7        2           0.058824
         10          0.666667
9        2           0.490196
         10          0.000000
14       2           0.000000
         10          0.000000